# Image Classification: EuroSAT Land Use and Land Cover Classification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukas-gysin/bverl-miniproject/blob/main/notebooks/00-submission.ipynb)

Submission of the group **GeoGuesser**
by Julian Steinacher and Lukas Gysin

## Introduction

## Data

## Models

## Results

## Discussion